In [ ]:
import time, sys, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import logging
from pathlib import Path
import random

sys.path.append('..')
sys.path.append('../../ocelot')
import pyiota.acnet as acnet
import pyiota.iota as iota
from pyiota.acnet.frontends import BPMDevice, BPMDeviceSet, DoubleDevice, DoubleDeviceSet, ACL, ACNETRelay, DPM, READ_METHOD

RING = iota.run4

proxy = ACNETRelay(set_multi=True, verbose=True)
proxy_cached = ACNETRelay(set_multi=True, verbose=True, read_method=READ_METHOD.CACHED)
acl = ACL(fallback=True)
acl_async = ACL()
dpm = DPM()

l = logging.getLogger('pyiota.acnet.acsys')
l.setLevel(logging.WARNING)

acnet.frontends.default_adapter = proxy

In [ ]:
ping = proxy.ping()
time.sleep(0.1)
ping = proxy.ping()
print(f'Proxy ping: {ping*1e3:.3f}ms')

In [ ]:
def get_ds_double(name, adapter):
    device = DoubleDevice(name)
    device_set = DoubleDeviceSet(name='test', members=[device], adapter=adapter)
    return device_set

In [ ]:
ds = get_ds_double('Z:ACLTST', dpm)
for i in range(20):
    ds.read()

In [ ]:
# p = proxy.start_proxy()
# time.sleep(10)
# p
ds = get_ds_double('Z:ACLTST', proxy)
for i in range(20):
    ds.read()

In [ ]:
# ds = get_ds('Z:ACLTST', acl)
# for i in range(10):
#     ds.read()

## BPMs

In [ ]:
def get_ds(adapter):
    bpms = [BPMDevice(b) for b in RING.BPMS.HA]
    ds_bpm = BPMDeviceSet(name='bpms', members=bpms, adapter=adapter)
    ds_bpm.array_length = 2048 # DPM requires array length or return only first value?
    return ds_bpm

In [ ]:
d = BPMDevice('N:IBC1RH')
arr = d.read()
print(len(arr))

In [ ]:
ds = get_ds(dpm)
for i in range(10):
    data = ds.read()
    for v in data:
        assert len(v) == 2048

In [ ]:
data[0][0:5], data[0][-5:]

In [ ]:
ds = get_ds(proxy)
for i in range(10):
    data = ds.read()
    for v in data:
        assert len(v) == 2048

In [ ]:
data[0][0:5], data[0][-5:]

In [ ]:
len(data[0])

## Write benchmarking

In [ ]:
ds = get_ds_double('Z:ACLTST', proxy)
for i in range(1):
    ds.read()
for i in range(1):
    ds.set([random.random()*5], verbose=True)

In [ ]:
            # /*
            # if (iadr.getHostAddress().startsWith("169.254")) {
            # 	System.out.println("Your node's java.net.InetAddress.getLocalHost().getHostAddress() is: " + java.net.InetAddress.getLocalHost().getHostAddress());
            # 	System.out.println("DaqUser, connections where java.net.InetAddress().getLocalHost().getHostAddress() start with 169.254 are not allowed.");
            # 	System.out.println("Addresses that begin with 131.225 are typically required.  Try disabling your wireless.");
            # 	return false;
            # }
            #  */